In [1]:
# Imports from our QAOA package
from forest_qaoa.qaoa.parameters import StandardParams, ExtendedParams, AbstractParams
from pyquil.paulis import PauliSum, PauliTerm
from pyquil.api import local_qvm, WavefunctionSimulator
from forest_qaoa.qaoa.cost_function import QAOACostFunctionOnWFSim
from scipy.optimize import minimize

In [2]:
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2 + (-0.5)*Z0")
print("hamiltonian =", hamiltonian)

timesteps = 1

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0


Set up two different sets of extended parameters:

In [3]:
extended1 = ExtendedParams.linear_ramp_from_hamiltonian(hamiltonian, timesteps)
extended2 = ExtendedParams.linear_ramp_from_hamiltonian(hamiltonian, timesteps)

The address of their `x_rotation_angles` always seems to be identical. The same is true sometimes, but not always, for the `z_rotation_angles` and the `zz_rotation_angles`

In [4]:
print(id(extended1.x_rotation_angles))
print(id(extended2.x_rotation_angles))

140015931803888
140015931804768


In [5]:
print(id(extended1.z_rotation_angles))
print(id(extended2.z_rotation_angles))

140015931805008
140015931805248


In [6]:
print(id(extended1.zz_rotation_angles))
print(id(extended2.zz_rotation_angles))

140015931804288
140015931805408


This becomes an issue when we define different sets of parameters in the same notebook, and then an optimiser updates the `x_rotation_angles` of one of the sets: naively we'd think it shouldn't affect the other at all, but since the memory location is the same, it does affect the second set.

<font color="red">
There appears to be a misunderstanding here. Even though `extended1.z_rotation_angles` appears to be an array, it actually is a property that is computed on the fly. Let's have a look at its source:
</font>

In [17]:
extended1.z_rotation_angles??

Type:        property
String form: <property object at 0x7f58019cc228>
Source:     
# extended1.z_rotation_angles.fget
@property
def z_rotation_angles(self):
    return self.single_qubit_coeffs * self.gammas_singles


<font color="red">
Nevertheless, the beheaviour seen below isn't the one intended. But it wasn't caused by `z_rotation_angles` being a descriptor. Instead it was caused by `gammas_singles` being a wrongly implemented data descriptor. I had implemented it in such a way, that all `gammas_singles` attributes of different instances of `ExtendedParams` were pointing at the same data. This is fixed now.
</font>

<font color='red'>
This isn't super surprising though.  They work as follows:
</font>

    
```python
class ExtendedParams(AbstractParams):
    def __init__():
        # bla
    
    # a lot of code
    
    @property
    def z_rotation_angles(self):
        return np.outer(self.gammas_singles, self.single_qubit_coeffs)
```
and we can access them, but not change them!
```python
z_angles = extended_params.z_rotation_angles

extended_params.z_rotation_angles = some_new_angles # <- throws an exception!
```


and now we can do

```python

z_angles = params.z_rotation_angles

```

## Example

In [7]:
sim = WavefunctionSimulator()

costfn1 = QAOACostFunctionOnWFSim(hamiltonian,
                                  params=extended1,
                                  sim=sim,
                                  scalar_cost_function=True, 
                                  nshots=1,                  
                                  noisy=False,               
                                  enable_logging=False)

costfn2 = QAOACostFunctionOnWFSim(hamiltonian,
                                  params=extended2,
                                  sim=sim,
                                  scalar_cost_function=True, 
                                  nshots=1,                  
                                  noisy=False,               
                                  enable_logging=False)

Print the two sets of `x_rotation_angles` before running the optimiser:

In [8]:
print('extended1 before', extended1.x_rotation_angles)
print('extended2 before', extended2.x_rotation_angles)

extended1 before [[0.35 0.35 0.35]]
extended2 before [[0.35 0.35 0.35]]


In [9]:
res1 = minimize(costfn1, extended1.raw(), tol=1e-3,
                      options={"maxiter": 500})

... and now after running the optimiser:

In [10]:
print('extended1 after', extended1.x_rotation_angles)
print('extended2 after', extended2.x_rotation_angles)

extended1 after [[4.83483169e-05 7.85112557e-01 7.85294709e-01]]
extended2 after [[0.35 0.35 0.35]]


Despite the fact we have not called the optimiser on `costfn2` yet, its associated angles have been modified by the optimiser call on `costfn1`.

<font color="red">
    Not anymore. It was fixed in this commit.
</font>